# Model Metadata API Demo

The Model Metadata API is still in developement. This demo showcases some of the functionality of the new API.

**Note**: If you are running with self-signed certificate make sure to set `CURL_CA_BUNDLE` environmental variable before executing this notebook, for example start the notebook server with:
```bash
CURL_CA_BUNDLE="" jupyter lab
```

## Api Configuration

In [1]:
from seldon_deploy_sdk.rest import ApiException

from seldon_deploy_sdk import V1Model, ModelMetadataServiceApi, Configuration, ApiClient
from seldon_deploy_sdk.auth import OIDCAuthenticator

In [2]:
from pprint import pprint
import json

In [3]:
# For local testing use
INGRESS = "http://localhost:8080"

# For production usecase set this to your ingress
# for example if your SD URL is http://xxx.yyy.zzz.xyz/seldon-deploy set
# INGRESS = "http://xxx.yyy.zzz.xyz"

In [5]:
config = Configuration()
config.host = INGRESS

In [6]:
# # auth if needed
# config.oidc_server = f"{INGRESS}/auth/realms/deploy-realm"
# config.oidc_client_id = "sd-api"
# config.username = "admin@seldon.io"
# config.password = "12341234"

# auth = OIDCAuthenticator(config)
# config.access_token = auth.authenticate()

In [7]:
api_client = ApiClient(config)
api_instance = ModelMetadataServiceApi(api_client)

## Add single model to the Model Catalogue

In [8]:
model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    artifact_type="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-test-model-alpha-repo",
        "an arbitrary tag": "true",
    },
)
try:
    # Create a Model Metadata entry.
    api_response = api_instance.model_metadata_service_create_model_metadata(model)
except ApiException as e:
    print(f"Couldn't create model: {json.loads(e.body)['message']}")

## Add multiple models to the Model Catalogue

In [9]:
models = [
    #     Same model different versions
    {
        "uri": "gs://test-model-beta-v1.0.0",
        "name": "beta",
        "version": "v1.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Jon"},
    },
    {
        "uri": "gs://test-model-beta-v2.0.0",
        "name": "beta",
        "version": "v2.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
    {
        "uri": "gs://test-model-beta-v3.0.0",
        "name": "beta",
        "version": "v3.0.0",
        "artifact_type": "SKLEARN",
        "task_type": "classification",
        "tags": {"author": "Bob"},
    },
]

for model in models:
    body = V1Model(**model)
    try:
        api_response = api_instance.model_metadata_service_create_model_metadata(body)
    except ApiException as e:
        print(f"Couldn't create model: {json.loads(e.body)['message']}")

## List all models in the Model Catalogue

In [10]:
try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata()
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'models': [{'artifact_type': 'UNKNOWN',
             'creation_time': datetime.datetime(2021, 4, 21, 13, 46, 58, 643917, tzinfo=tzutc()),
             'metrics': {},
             'name': '',
             'tags': {'auto_created': 'true'},
             'task_type': '',
             'uri': 'gs://seldon-models/tfserving/cifar10/resnet32',
             'version': ''},
            {'artifact_type': 'UNKNOWN',
             'creation_time': datetime.datetime(2021, 4, 21, 13, 46, 58, 652352, tzinfo=tzutc()),
             'metrics': {},
             'name': '',
             'tags': {'auto_created': 'true'},
             'task_type': '',
             'uri': 'gs://seldon-models/sklearn/income/model-0.23.2',
             'version': ''},
            {'artifact_type': 'UNKNOWN',
             'creation_time': datetime.datetime(2021, 4, 21, 13, 46, 58, 657373, tzinfo=tzutc()),
             'metrics': {},
             'name': '',
             'tags': {'auto_created': 'true'},
             'task_type': 

## Get all version of a given model (named "beta")

In [11]:
# uri = 'uri_example'                       # str |  (optional)
# name = 'name_example'                     # str |  (optional)
# version = 'version_example'               # str |  (optional)
# artifactType = 'artifactType_example' # str |  (optional)
# task_type = 'task_type_example'           # str |  (optional)
# model_type = 'model_type_example'         # str |  (optional)

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(name="beta", tags={"author": "Jon"})
    print("Filter by name=beta")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Filter by name=beta
{'models': [{'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2021, 4, 21, 13, 47, 8, 377704, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'tags': {'author': 'Jon'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}


## Get all models authored by Bob (tags.author = Bob)

In [12]:
# uri = 'uri_example'                       # str |  (optional)
# name = 'name_example'                     # str |  (optional)
# version = 'version_example'               # str |  (optional)
# artifactType = 'artifactType_example' # str |  (optional)
# task_type = 'task_type_example'           # str |  (optional)
# model_type = 'model_type_example'         # str |  (optional)

try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(tags={"author": "Bob"})
    print("Filter by name=beta")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Filter by name=beta
{'models': [{'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2021, 4, 21, 13, 47, 8, 396714, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'tags': {'author': 'Bob'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v2.0.0',
             'version': 'v2.0.0'},
            {'artifact_type': 'SKLEARN',
             'creation_time': datetime.datetime(2021, 4, 21, 13, 47, 8, 417363, tzinfo=tzutc()),
             'metrics': {},
             'name': 'beta',
             'tags': {'author': 'Bob'},
             'task_type': 'classification',
             'uri': 'gs://test-model-beta-v3.0.0',
             'version': 'v3.0.0'}],
 'next_page_token': ''}


## Modify model metadata entry in the Model Catalogue

In [13]:
try:
    # Get Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(uri="gs://test-model-alpha-v1.0.0")
    print("Before update:")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")


model = V1Model(
    uri="gs://test-model-alpha-v1.0.0",
    name="alpha",
    version="v1.0.0",
    artifact_type="XGBOOST",
    task_type="regression",
    tags={
        "source": "https://github.com/some-other-repo",
        "an arbitrary tag": "true",
        "an additional tag": "123",
    },
)

try:
    # Update a Model Metadata entry.
    api_response = api_instance.model_metadata_service_update_model_metadata(model)
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")
    
try:
    # List Model Metadata entries.
    api_response = api_instance.model_metadata_service_list_model_metadata(
        uri="gs://test-model-alpha-v1.0.0"
    )
    print("After update:")
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

Before update:
{'models': [{'artifact_type': 'XGBOOST',
             'creation_time': datetime.datetime(2021, 4, 21, 13, 47, 8, 353363, tzinfo=tzutc()),
             'metrics': {},
             'name': 'alpha',
             'tags': {'an arbitrary tag': 'true',
                      'source': 'https://github.com/some-test-model-alpha-repo'},
             'task_type': 'regression',
             'uri': 'gs://test-model-alpha-v1.0.0',
             'version': 'v1.0.0'}],
 'next_page_token': ''}
{}
After update:
{'models': [{'artifact_type': 'XGBOOST',
             'creation_time': datetime.datetime(2021, 4, 21, 13, 47, 8, 353363, tzinfo=tzutc()),
             'metrics': {},
             'name': 'alpha',
             'tags': {'an additional tag': '123',
                      'an arbitrary tag': 'true',
                      'source': 'https://github.com/some-other-repo'},
             'task_type': 'regression',
             'uri': 'gs://test-model-alpha-v1.0.0',
             'version': 'v1.0

## Get runtime information for a model

In [14]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = api_instance.model_metadata_service_list_runtime_metadata_for_model(
        model_uri="gs://seldon-models/sklearn/iris", 
        deployment_status="Running"
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'next_page_token': '',
 'runtime_metadata': [{'creation_time': datetime.datetime(2021, 4, 21, 13, 46, 58, 658160, tzinfo=tzutc()),
                       'deployment_kubernetes_uid': '50795fc5-79b5-49b1-b01b-fbe95661fbc2',
                       'deployment_name': 'iris',
                       'deployment_namespace': 'seldon',
                       'deployment_status': 'Running',
                       'deployment_type': 'SeldonDeployment',
                       'model_uri': 'gs://seldon-models/sklearn/iris',
                       'node_name': 'iris-container',
                       'predictor_name': 'default',
                       'shadow': False,
                       'traffic': '100'},
                      {'creation_time': datetime.datetime(2021, 4, 21, 13, 46, 58, 662457, tzinfo=tzutc()),
                       'deployment_kubernetes_uid': 'fde71324-823c-46b5-9971-f45239df60b8',
                       'deployment_name': 'iris-2',
                       'deployment_namesp

## Get model information for a deployment

In [15]:
try:
    # List Runtime Metadata for all deployments associated with a model.
    api_response = api_instance.model_metadata_service_list_runtime_metadata_for_model(
        deployment_name="iris", 
        deployment_namespace="seldon"
    )
    pprint(api_response)
except ApiException as e:
    print(f"Failed to call API: {json.loads(e.body)['message']}")

{'next_page_token': '',
 'runtime_metadata': [{'creation_time': datetime.datetime(2021, 4, 21, 13, 46, 58, 658160, tzinfo=tzutc()),
                       'deployment_kubernetes_uid': '50795fc5-79b5-49b1-b01b-fbe95661fbc2',
                       'deployment_name': 'iris',
                       'deployment_namespace': 'seldon',
                       'deployment_status': 'Running',
                       'deployment_type': 'SeldonDeployment',
                       'model_uri': 'gs://seldon-models/sklearn/iris',
                       'node_name': 'iris-container',
                       'predictor_name': 'default',
                       'shadow': False,
                       'traffic': '100'}]}
